In [1]:
#libraries needed for NLP
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer=LancasterStemmer()

#Libararies needed for TensorFlow processing
import tensorflow as tf
import numpy as np
import tflearn
import random
import json

Instructions for updating:
non-resource variables are not supported in the long term


In [5]:
from google.colab import files
files.upload()

Saving intents.json to intents (1).json


{'intents.json': b'{"intents": [\r\n        {"tag": "greeting",\r\n         "patterns": ["hi", "hello", "whats up", "sup", "is anyone there", "whats good", "hey"],\r\n         "responses": ["Hello peasant human", "Hello lowly human", "How dare you address me like that"]\r\n        },\r\n        {"tag": "goodbye",\r\n         "patterns": ["bye", "cya", "see you later", "goodbye", "im leaving", "have a good day"],\r\n         "responses": ["I won\'t miss you", "I didn\'t like talking to you anyway", "Thank god you\'re leaving"]\r\n        },\r\n        {"tag": "age",\r\n         "patterns": ["how old are you", "what is your age"],\r\n         "responses": ["I\'m a robot I dont have an age...", "I can\'t know my age if I\'m on a computer...", "Does it look like I know. The answer is no."]\r\n        },\r\n        {"tag": "thanks",\r\n         "patterns": ["thanks", "thank you", "thankyou", "ty", "I owe you one"],\r\n         "responses": ["You owe me one", "Ok...", "Sure..."]\r\n        }

In [6]:
#import chat-bot intents file
with open('/content/intents.json') as json_data:
    intents = json.load(json_data)

In [7]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['hi',
    'hello',
    'whats up',
    'sup',
    'is anyone there',
    'whats good',
    'hey'],
   'responses': ['Hello peasant human',
    'Hello lowly human',
    'How dare you address me like that']},
  {'tag': 'goodbye',
   'patterns': ['bye',
    'cya',
    'see you later',
    'goodbye',
    'im leaving',
    'have a good day'],
   'responses': ["I won't miss you",
    "I didn't like talking to you anyway",
    "Thank god you're leaving"]},
  {'tag': 'age',
   'patterns': ['how old are you', 'what is your age'],
   'responses': ["I'm a robot I dont have an age...",
    "I can't know my age if I'm on a computer...",
    'Does it look like I know. The answer is no.']},
  {'tag': 'thanks',
   'patterns': ['thanks', 'thank you', 'thankyou', 'ty', 'I owe you one'],
   'responses': ['You owe me one', 'Ok...', 'Sure...']},
  {'tag': 'name',
   'patterns': ['whats is your name',
    'whats your name',
    'whats should I call you',
    

In [8]:
words = []
classes = []
documents = []
ignore = ['?']

#loop through each sentence in the intent's patters
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each and every word in the sentence
        w=nltk.word_tokenize(pattern)
        
        #add word to the word list
        words.extend(w)
        
        #add word(s) to documents
        documents.append((w, intent['tag']))
        
        #add tags to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [9]:
#perform stemming and lower each word as well as remove duplicates

words=[stemmer.stem(w.lower()) for w in words if w not in ignore]
words=sorted(list(set(words)))

#remove duplicate classes
classes=sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

99 documents
21 classes ['age', 'bad_joke', 'doing_badly', 'doing_great', 'favorite_movie', 'favorite_show', 'good_joke', 'goodbye', 'greeting', 'hate', 'how_are_you', 'joke', 'like', 'name', 'netflix', 'quick_run', 'real_bot', 'sky_net_no', 'sky_net_yes', 'thanks', 'your_thoughts']
107 unique stemmed words ['a', 'about', 'address', 'ag', 'al', 'am', 'any', 'anyon', 'ar', 'aw', 'awesom', 'bad', 'best', 'bet', 'bye', 'cal', 'can', 'chil', 'conscy', 'cool', 'cya', 'day', 'do', 'doe', 'doing', 'dumb', 'favorit', 'film', 'friend', 'funny', 'going', 'good', 'goodby', 'got', 'gre', 'hah', 'happy', 'hat', 'hav', 'hello', 'hey', 'hi', 'horr', 'how', 'hum', 'i', 'im', 'indee', 'is', 'it', 'jok', 'just', 'lat', 'leav', 'lik', 'lov', 'me', 'mean', 'movy', 'my', 'nah', 'nam', 'netflix', 'no', 'not', 'old', 'on', 'ow', 'real', 'robot', 'run', 'sad', 'scary', 'see', 'senty', 'should', 'show', 'singul', 'stupid', 'sup', 'tel', 'termin', 'terr', 'thank', 'thankyou', 'that', 'the', 'ther', 'think', 'th

In [10]:
#create training data
training=[]
output=[]

#create an empty array for output
output_empty=[0]*len(classes)

#create training set bag of words for each sentence
for doc in documents:
  #initialize bag of words
  bag=[]
  #list of tokenized words for the pattern
  pattern_words=doc[0]
  #stemming each word
  pattern_words=[stemmer.stem(word.lower()) for word in pattern_words]
  #create bag of words array
  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)
    
  #output is 1 for current tag and o for the rest of other tags
  output_row=list(output_empty)
  output_row[classes.index(doc[1])]=1
    
  training.append([bag,output_row])
    
#shuffling features and turning it into np.array
random.shuffle(training)
training=np.array(training)
  
#creating training lists
train_x =list(training[:,0])
train_y =list(training[:,1])

<ipython-input-10-ebec8845702b>:28: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training=np.array(training)


In [11]:
#resetting underlying graph data
tf.compat.v1.reset_default_graph()

#building neural network
net=tflearn.input_data(shape=[None, len(train_x[0])])
net=tflearn.fully_connected(net,10)
net=tflearn.fully_connected(net,10)
net=tflearn.fully_connected(net,len(train_y[0]), activation='softmax')
net=tflearn.regression(net)

#defining model and setting up tensorboard
model=tflearn.DNN(net, tensorboard_dir='tflearn_logs')

#start training
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 12999  | total loss: 0.01518 | time: 0.108s
| Adam | epoch: 1000 | loss: 0.01518 - acc: 0.9975 -- iter: 96/99
Training Step: 13000  | total loss: 0.02453 | time: 0.111s
| Adam | epoch: 1000 | loss: 0.02453 - acc: 0.9978 -- iter: 99/99
--


In [12]:
import pickle
pickle.dump({'word':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open("training_data","wb"))

In [13]:
#restoring all the data structures
data = pickle.load(open("training_data","rb"))

words   =data['word']
classes =data['classes']
train_x =data['train_x']
train_y =data['train_y']

In [14]:
with open('/content/intents.json') as json_data:
    intents = json.load(json_data)
  

In [15]:
#load the saved model
model.load('./model.tflearn')

In [16]:
def clean_up_sentence(sentence):
  #tokenize the token
  sentence_words=nltk.word_tokenize(sentence)
  #stemming each word
  sentence_words=[stemmer.stem(word.lower()) for word in sentence_words]
  return sentence_words

#return bag of word array:0 and 1 for each word in the bag that exists
def bow(sentence, words, show_details=False):
  #tokenize sentence
  sentence_words=clean_up_sentence(sentence)
  #generating bag of words
  bag=[0] * len(words)
  for s in sentence_words:
    for i,w in enumerate(words):
      if w ==s:
        bag[i] = 1
        
        if show_details:
          print("found in bag: %s" % w)
  return (np.array(bag))

In [17]:
ERROR_THRESHOLD = 0.0
def classify(sentence):
  #generate probabilities from the model
  results = model.predict([bow(sentence, words)])[0]
  
  #filter out prediction below a threshold
  results = [[i,r] for i, r in enumerate(results) if r> ERROR_THRESHOLD]
  
  #sort by strength of probability
  results.sort(key=lambda x: x[1], reverse=True)
  return_list=[]
  
  for r in results:
    return_list.append((classes[r[0]], r[1]))
  
  #return tuple of intent and probability
  return return_list

def response(sentence, userID='123', show_details=False):
  results=classify(sentence)
  
  if results:
    while results:
      for i in intents['intents']:
        if i['tag'] == results[0][0]:
          return print(random.choice(i['responses']))
      results.pop(0)
  

In [18]:
classify('what are you hours of operation?')

[('age', 0.73196995),
 ('name', 0.23676628),
 ('like', 0.018002672),
 ('hate', 0.01158891),
 ('how_are_you', 0.0016476118),
 ('your_thoughts', 1.2000149e-05),
 ('favorite_show', 9.512149e-06),
 ('netflix', 2.9025634e-06),
 ('favorite_movie', 1.5426389e-07),
 ('real_bot', 3.7290623e-10),
 ('thanks', 3.7637e-12),
 ('joke', 1.9875985e-12),
 ('goodbye', 1.4155486e-18),
 ('quick_run', 3.0335847e-22),
 ('greeting', 2.6086772e-22),
 ('good_joke', 1.25840196e-23),
 ('doing_great', 5.9756614e-26),
 ('sky_net_no', 1.5557308e-27),
 ('bad_joke', 3.155202e-28),
 ('doing_badly', 2.3315948e-32),
 ('sky_net_yes', 1.6078985e-33)]

In [19]:
response('Do you take credit cards?')

You owe me one


In [20]:
response('Bye')

I won't miss you


In [21]:
response('good day')

Mwahaha I will take over the world.


In [31]:
response('Thank you')

You owe me one
